### timer - Hardware timer classes

In real time firmware, hardware timers are necessary and must be as fast as possible.  I've interfaced the ``c`` API for timers in a signal/slot oriented way in ``ah_timer`` as well as simply, in ``simple_timer``.  these use ``sigevents`` and ``itimerspec`` both to generate the timer.

#### Example

Below are two extremely simple examples on how to use my timers.

In [1]:
from cppm import cppmagic

Both of these timers require the ``-lrt`` (library real time) option, so make sure you put that at the end of your compiler.

In [23]:
%set_compiler g++ --std=c++11 -lrt

The first example is my ``simple_timer``.  This is good for just doing one single task, and timing it.  It's pretty accurate (to within $1\mathrm{ms}$ in my experience), but has a couple problems:

- it's currently not working on OSX
- you can only define one of them
- you can only use the ``SIGALRM`` signal (at least in my tests)

I'll be working on these problems, but until then, you should probably use my ``ah_timer`` class, instead.  The example of that in below this one.

But, this library is totally usable.  Basically we have to create a timer, a function for when the timer is caught, and connect it to a signal.  Pretty straightforward.

In [24]:
%%runcpp simple_timer_test
#include <stdio.h>
#include "../simple_timer.h"
#include "../ah_signal.h"
#include <unistd.h>

bool cont = true;
simple_timer* ub;

void usain_bolt(int signum){
    ub->stop();
    printf("Usain Bolt finished 100 meters.\n");
    cont = false;
}

int main(void){
    ub = new simple_timer();
    ah_signal(SIGALRM, &usain_bolt);
    printf("Go!\n");
    ub->start(1.5);
    while(cont){
        printf(".\n");
        usleep(2.5E5);
    }
    printf("Time was %f\n",ub->get_t());

    return 0;
}

Go!
.
.
.
.
.
.
Usain Bolt finished 100 meters.
Time was 1.500116



The ``ah_timer`` class is a bit more complicated.  The example below shows me creating a class which has two timers.  Each timer is then connected to a class method which stops the itmer and prints its result.  Then, they are started using a thread.  To use the thread, we have to add ``-lpthread`` to our compiler.

So I start each timer and wait until it finishes, and then I can print out the results.  Note that ``usain_bolt`` is connected to an interval timer, which means he will continue running until the program quits or the destructor is called.  ``betty_white`` only runs once, because I used ``->set()``.  There's also the ability to add a different time to ``on_deck``, which I may update this example with in the future.

In [47]:
%set_compiler g++ --std=c++11 -lrt -lpthread

In [49]:
%%runcpp timer_test
#include "../timer.h"
#include "../common/sigslot.h"
#include <stdio.h>
#include <thread>
#include <string>
#include <unistd.h>

class timers : public sigslot::has_slots<>{
public:
	timer *ub,*bw;
    int ubcount = 0;
	timers();
	void usain_bolt(void);
	void betty_white(void);
	void print_ub(void);
	void print_bw(void);
};

timers::timers(void){
	std::thread ub(&timers::usain_bolt, this);
	std::thread bw(&timers::betty_white, this);

	ub.join();
	bw.join();
}

void timers::usain_bolt(void){
	ub = new timer();
	ub->sig->connect(this, &timers::print_ub);
	ub->set_int(1.50);
	ub->start();
}
void timers::betty_white(void){
	bw = new timer();
	bw->sig->connect(this, &timers::print_bw);
	bw->set(5.00);
	bw->start();
}

void timers::print_ub(void){
    ub->finish();
    ubcount++;
    std::string howmany;
    if (ubcount > 1) {
        howmany = "more";
    } else {
        howmany = "";
    }
	printf("Usain Bolt finished 100 %s meters in %f seconds.\n", howmany.c_str(), ub->realtime);
}
void timers::print_bw(void){
    bw->finish();
	printf("Betty White finished 100 meters in %f seconds.\n", bw->realtime);
}

int main(void){
    timers *t = new timers();
    printf("Go!\n");
    int i = 0;
    while (i < 9){
        printf(".\n");
        usleep(1E6);
        i++;
    }
    return 0;
}

Go!
.
.
Usain Bolt finished 100  meters in 1.500175 seconds.
.
.
Usain Bolt finished 100 more meters in 1.499955 seconds.
.
.
Usain Bolt finished 100 more meters in 1.499977 seconds.
.
Betty White finished 100  meters in 5.000081 seconds.
.
Usain Bolt finished 100 more meters in 1.499929 seconds.
.

